<a href="https://colab.research.google.com/github/ElianGonzalez0202/Cart.2.0/blob/main/Copia_de_Cart_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 31005_A2: Programación del algoritmo del árbol de decisión (CART)

Hong Kung(12431868) & Weixiang Gao(12765653)

Video link: https://www.youtube.com/watch?v=koAg_y_l3VY&feature=youtu.be
<br>


## Introducción



El análisis de datos mediante la construcción de modelos se ha utilizado ampliamente en diversos campos, especialmente para analizar los diferentes aspectos de los bienes en el mercado. El diamante es uno de los productos de inversión más populares en los últimos años, y su rango de precios es muy amplio, lo que se debe a sus diferencias de calidad en muchos aspectos.

Por lo tanto, el objetivo de nuestro proyecto es simple: programar un árbol de decisión para clasificar el precio de los diamantes (bajo, medio, alto). Para lograrlo, utilizamos el conjunto de datos **diamonds.csv** de Kaggle para entrenar y probar nuestro modelo de árbol de decisión. La opción más adecuada en este caso es un árbol binario de estructura simple. Optamos por el Árbol de Clasificación y Regresión (CART). Dado que todos los datos utilizados son discretos, se creará un árbol de clasificación en lugar de uno de regresión. Los detalles del algoritmo del árbol y la construcción del modelo se analizarán en la siguiente sección.

La entrada de nuestro proyecto es el conjunto de datos de diamantes, utilizado para el entrenamiento y las pruebas. El resultado es el árbol de decisión CART y su precisión de prueba. Y por último hablaremos del problema ético relacionado con nuestro proyecto.

## Exploración

### Identificar el desafío

El desafío principal de este proyecto es que CART solo admite división binaria, lo que significa que los datos solo se pueden dividir en dos partes (sí, respuesta al divisor, no respuesta al divisor); el desafío principal es diseñar el modelo de estos problemas y determinar los divisores bajo la teoría CART básica.


### Preparación

Para iniciar nuestro programa, necesitamos importar todas las bibliotecas que necesitamos para este proyecto.

In [43]:
# procesamiento de matriz estándar
import numpy as np
# Procesamiento de marcos de datos
import pandas as pd

# para seleccionar aleatoriamente datos de entrenamiento/prueba del marco de datos
import random
# "Imprimir de forma bonita" una estructura de datos arbitraria (en nuestro caso, un objeto dict de un árbol) de forma más clara.
from pprint import pprint

Luego, carguemos y verifiquemos el marco de datos de origen.

In [19]:
url = 'https://raw.githubusercontent.com/STKKKKK/UTS_ML_2019_ID12431868/master/31005_ML_A2/diamonds.csv'

df = pd.read_csv(url)
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


El marco de datos contiene los siguientes atributos:<br>
<br>
Precio: precio en dólares estadounidenses (326-18.823 dólares)

Quilate: peso en quilates (0,2-5,01 dólares)

Talla: calidad de la talla (Regular, Buena, Muy Buena, Premium, Ideal)

Color: color del diamante, de J (peor) a D (mejor)

Claridad: medida de la claridad del diamante: I1 (peor), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (mejor)

x: longitud en mm (0-10,74 dólares)

y: ancho en mm (0-58,9 dólares)

z: profundidad en mm (0-31,8 dólares)

Profundidad: porcentaje de profundidad total = z / media (x, y) = 2 * z / (x + y) (43-79 dólares)

Tabla : ancho de la parte superior del diamante en relación con el punto más ancho (43--95)

In [20]:
#  una visión general

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


Y afortunadamente, no hay valores nulos en el conjunto de fechas.

De ahora en adelante, debemos decidir qué atributos son más significativos en este proyecto. Según el estándar oficial de clasificación de diamantes (GIA), los cuatro factores más importantes (4C) son: peso en quilates, talla, color y claridad. <br>
Por lo tanto, conservaremos estos atributos y eliminaremos las demás columnas del marco de datos.

In [21]:
df = df.drop(['Unnamed: 0','depth','table','x','y','z'],axis=1)

Además, podemos convertir fácilmente los datos categóricos a ***valores numéricos***, ya que estos datos son "escalas" de un diamante. Según el GIA de diamantes, utilizamos la escala de clasificación completa.

In [22]:
grade = {'cut': ['Fair','Good','Very Good','Premium','Ideal'],
         'color': ['J','I','H','G','F','E','D'],
         'clarity': ['I3','I2','I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF','FL']}
         # ordenados de peor a mejor

for key, scale in grade.items():
   for index in range(len(scale)):
      df[key] = df[key].map(lambda x: index if x == grade[key][index] else x)

Por supuesto, como mencionamos, también necesitamos convertir el precio en dólares estadounidenses a los grados bajo, medio y alto. Antes de eso, necesitamos hacer un resumen general de la distribución del valor del precio.

In [23]:
df.describe()

,carat,cut,color,clarity,price
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.904097,3.405803,5.051020,3932.799722
std,0.474011,1.116600,1.701105,1.647136,3989.439738
min,0.200000,0.000000,0.000000,2.000000,326.000000
25%,0.400000,2.000000,2.000000,4.000000,950.000000
50%,0.700000,3.000000,3.000000,5.000000,2401.000000
75%,1.040000,4.000000,5.000000,6.000000,5324.250000
max,5.010000,4.000000,6.000000,9.000000,18823.000000


En este caso, tomamos el valor de 1200 y 4000 como valor límite de la clase de precio.

In [24]:
df['price'] = df['price'].map(lambda x: 'low' if x<1200 else
                              ('high' if x>4000 else 'medium'))

Ahora, verifiquemos finalmente los datos que se utilizaron únicamente para este programa.

In [25]:
df.head()

,carat,cut,color,clarity,price
0,0.23,4,5,3,low
1,0.21,3,5,4,low
2,0.23,1,5,6,low
3,0.29,3,1,5,low
4,0.31,1,0,3,low


Además, tenemos que decidir aleatoriamente los índices de los marcos de datos de entrenamiento y prueba, obtendremos 200 muestras para cada uno.

In [26]:
random.seed(1)

indices = df.index.tolist()
test_indices = random.sample(population=indices, k=200)

test_df = df.loc[test_indices]
train_df = df.drop(test_indices)

Finalmente, convertiremos el marco de datos del tren en una matriz 2D numpy para un procesamiento más conveniente y lo asignaremos a la variable 'datos'.

In [27]:
data = train_df.values

Tenga en cuenta que debemos tener estas variables que usaríamos con frecuencia en nuestro programa:<br>
- df
- test_df
- train_df
- data

### Modelado

 ![flowchart](https://raw.githubusercontent.com/STKKKKK/UTS_ML_2019_ID12431868/master/31005_ML_A2/A2_flowChart.png)

Según el diagrama de flujo, se muestra que es necesario lograr un bucle de división de datos. Por lo tanto, se utilizará un árbol de decisión (CART) para gestionar esto. Por supuesto, los árboles de decisión ID3 y C4.5 también pueden ser enfoques alternativos. <br>
El concepto clave del árbol de decisión CART es identificar la división con base en el índice de Gini, que es un valor que describe la impureza de un conjunto de datos. Cuanto menor sea el índice de Gini de un conjunto de datos, mayor será la probabilidad de que los datos pertenezcan a la misma clase.<br>
En este caso, necesitamos encontrar la mejor división entre todas las divisiones potenciales calculando la ganancia de Gini más baja de estas divisiones.<br>
Como también hemos observado que el término ***'división' o 'divisor'*** se menciona con frecuencia, es una buena manera de definirlo como un objeto abstracto.

## Metodología

### un objeto SPLIT

In [42]:
class Split():

    def __init__(self, data, column, value):
        """
        Columna: Atributo SPLIT (índice de datos de la columna)
Valor: Valor de SPLIT
Datos por debajo/por encima: Conjunto de datos que se divide en SPLIT
        """
        self.column = column
        self.value = value
        self.data_below = data[data[:, self.column] < value]
        self.data_above = data[data[:, self.column] >= value]


    def formatt(self):
        """
        Representar el objeto SPLIT de una manera más intuitiva
        """
        return "{} < {}".format(df.columns[self.column], self.value)


    def gini_gain(self):
        """
        Calcular la ganancia de Gini del SPLIT
          Considere que este valor utiliza el puntaje de Gini
          de ambos conjuntos de datos.
        """
        def gini(data):
            """
            Calcular el índice de Gini de un conjunto de datos
            """
                # Obtenga el recuento de grados de precio únicos
            _, counts = np.unique(data[:, -1], return_counts=True)
            p = counts / counts.sum()

                # Fórmula del índice de Gini:
            Gini = 1 - sum(p ** 2)

            return Gini

        n_data = len(self.data_below) + len(self.data_above)
        p_below = len(self.data_below) / n_data
        p_above = len(self.data_above) / n_data

            # Fórmula del coeficiente de Gini:
        Gini_gain = p_below*gini(self.data_below)+ p_above*gini(self.data_above)

        return Gini_gain

In [29]:
# pequeña prueba

Split(data, 3, 1.8).gini_gain()

np.float64(0.6655809650983266)

### Obtenga la mejor división

In [41]:
def get_best_split(data):
    """
    1. Inicialmente, necesitamos obtener todas las posibles divisiones del conjunto de datos.
Esto se puede lograr encontrando TODOS los diferentes puntos de datos en el conjunto de datos,
y la división tomará el valor promedio de cada dos puntos para que ambos puntos se ubiquen
lo más lejos posible de la división..

    2. Después de eso, encontraremos el mejor Split en función de su gini_gain.
    """
    potential_splits = []
    _, n_columns = data.shape

    for column in range(n_columns -1):   # we don't need the price column
        unique_values = np.unique(data[:, column])

        for index in range(len(unique_values)):
                # El primer valor no será procesado
                # porque no tiene un valor previo
            if index > 0:
                current = unique_values[index]
                previous = unique_values[index -1]
                split = Split(data, column, (current + previous)/2)
                potential_splits.append(split)


       # iterar desde todas las divisiones potenciales
       # un valor de gini_gain nunca será mayor que 1, por lo que podemos comenzar desde aquí
    max_gini = 1

    for split in potential_splits:
          if split.gini_gain() <= max_gini:
            max_gini = split.gini_gain()
            best_split = split

    return best_split

In [31]:
# Pequeña prueba: esta es la primera división de datos

sp = get_best_split(data)
sp.formatt()

'carat < 0.495'

### El árbol de decisiones

In [39]:
def train(data, max_depth, depth = 0):
    """
Esta función tenía como objetivo construir el árbol de decisión CART mediante el entrenamiento de los datos.
¡Es una función recursiva!
max_depth: se puede configurar para definir la profundidad máxima del árbol.

    El árbol será un objeto dict anidado, considerando que está formado por subárboles con tres nodos: {root: [true_leaf, false_leaf]}
    """
    price_column = data[:, -1]
    unique_classes, count = np.unique(price_column, return_counts=True)

        # Si el grado de precio es único o se alcanza la profundidad máxima establecida
    if(len(unique_classes) == 1) or (depth == max_depth):
           # clasificar
        end_node = unique_classes[count.argmax()]

        return end_node

    else:
        depth += 1    # cuenta hasta alcanzar la profundidad máxima

        best_split = get_best_split(data)

           # arbol de construccion
        root = best_split.formatt()
        sub_tree = {root: []}
        true_leaf = train(best_split.data_below, max_depth, depth)
        false_leaf = train(best_split.data_above, max_depth, depth)

           # Asegúrese de que la misma hoja no se represente al mismo tiempo.
        if true_leaf != false_leaf:
            sub_tree[root].append(true_leaf)
            sub_tree[root].append(false_leaf)
        else: sub_tree = true_leaf

        return sub_tree

In [36]:
def test(data, tree):
    """
    La función buscaba probar los datos mediante un árbol de decisión CART.
¡Esta es una función recursiva!
    """
    root = list(tree.keys())[0]    # get the root node of sub_tree
    column, less_than, value = root.split()

    if data[column] < float(value):   # 'value' needs to convert from str
        leaf = tree[root][0]
    else: leaf = tree[root][1]

    if type(leaf) == dict:    # still haven't reach the end node of whole tree
        return test(data, leaf)
    else: return leaf     # finally reach the end node

## Evaluación

### Actuación

**First, let's do a simple test to check if the tree testing function works:**

In [34]:
print(test_df.iloc[14],'\n')
print("The price is '{}'".format(test(test_df.iloc[14], tree)))

carat      1.03
cut           4
color         5
clarity       4
price      high
Name: 13759, dtype: object 



NameError: name 'tree' is not defined

The test seems work and we can find out that the 14th test (out of 200 samples) is a correct one.
<br>
<br>



**Now, we can Train the data and Create a decision tree with depth of 5.**

In [ ]:
tree = train(data, 5)

pprint(tree)  # use 'pretty-print'

**After that, we will apply the test samples to do the test. And finally, we can calculate the accuracy.**

In [ ]:
def accuracy(data, tree):

     # create a new column of test results
  test_df['test_result'] = test_df.apply(test, axis=1, args=(tree,))

     # calculate the mean of boolean values of correctness
     #  (1 if correct, 0 if wrong) , we get the Accuracy of the 'tree'
  accuracy = (test_df.test_result == test_df.price).mean()

  print(test_df.head())
  print('\n','\n','The Accuracy of our CART decision tree is {}'.format(accuracy))


In [ ]:
accuracy(test_df, tree)

<br>
<br>

**Let's Train and Test again with a tree of depth of 6.**

In [ ]:
tree_depth_6 = train(data, 6)
accuracy(test_df, tree_depth_6)

### Effiency

In [ ]:
%timeit tree_depth_of_4 = train(data, 4)

In [ ]:
%timeit tree_depth_of_5 = train(data, 5)

In [ ]:
%timeit tree_depth_of_6 = train(data, 6)

The overall time effiency is not as satisfied as accuracy, that's why we always need to limit the depth of the tree according to different situations. (bigger/deeper the tree, higher the accuracy but lower the effiency)

### Comparative Study

The main concept of CART algorithm is Gini-gain. In fact, there are two others types of widely-used decision tree that use the different concept and have its own limitation:
- ID3 decision tree is a greedy algorithm, decision making is based on information gain, by finding the split of the lowest overall entropy(a value descibe chaostic of information), which means the highest information gain.
 However ID3 decision tree can not deal with continous data.
- C4.5 decision tree is an advanced version of ID3, instead of finding information gain, it turned to find out the information gain ratio.
 C4.5 algorithm support both continous data and discrete data just like CART, but it still tend to fit discrete data better.
- CART is what we are programming in this project. The most significant limitation of CART is binary splis, this may lead to the data over-fitting issue.

We can also use the sklearn library to implement all decision tree algorithms. Note that we can also use the iris data to do the training and testing.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
import graphviz

iris = load_iris()
X = iris.data
y = iris.target
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,y)

 # dot data graph
data = tree.export_graphviz(clf, out_file=None,
                                 feature_names=iris.feature_names,
                                 class_names=iris.target_names,
                                 filled=True,
                                 rounded=True,
                                 special_characters=True)

graph = graphviz.Source(data)
graph.view()

## Conclusion

To conclude, within the limitation of time and knowledge, we learned how to overcome the difficulties during developing and felt very accomplished when the model built successfully and could achieve the goal of the project, it finds the price rank of a diamond mainly influenced by analysing the carat weight, cut, colour and clarity of a diamond. The accuracy is acceptable enough (about 0.95 when the tree depth is 5). Ofcourse, we can also figure how to build a regression CART and there are several improvements of current CART algorithm, such as how to handle with data over-fitting, as these require much deeper understanding of tree model structuring and the whole machine learning field.   

## Ethical

Ethical considerations are inevitable as computers are playing more and more important roles in humans’ life. There are different positions on ethics with the machine. Some might argue that machine ethics exists because humans are machines and humans have ethics. Others could argue that machine ethics doesn’t exist because ethics is simply emotional expression and machines can’t have emotions (Moor, 2006). Although the discussion of this may quickly bring us into philosophical issues, we cannot ignore it. we can’t—and shouldn’t—avoid consideration of machine ethics in today’s technological world, said by Moor, (2006). According to Kantian duty-based approach, it is not things that affect people, but people that influence things. It is our people who are constructing the real world. In the process of knowing things, people are more important than things themselves. People are morally autonomous. Although human behaviour is restricted by objective causality, people become human beings because they have moral freedom, can transcend cause and effect, and can be responsible for their actions. Therefore, it is important to determine the correct ethics, and ethics must have supreme authority. Ethics measure whether the action itself is in line with ethical standards, without it as a result of happiness. Considering that our project is the model to analyse the determining factors of the price of a diamond, it is a guide for the customers who are not professional in this field. It could help them to identify whether the diamonds they buy are worth the corresponding price. The market is determined by the relationship between supply and demand so our model will not destroy the balance. Within our knowledge, our project is unlikely to be misused.

## Video Pitch


https://www.youtube.com/watch?v=koAg_y_l3VY&feature=youtu.be

## Reference

LUMERA: The 4 C's Of Diamonds, & So Much More https://www.lumeradiamonds.com/diamond-education/index <br>
Moor, J. (2006). The Nature, Importance, and Difficulty of Machine Ethics. IEEE Intelligent Systems, 21(4), pp.18-21.<br>